In [1]:
import numpy as np
from tqdm import tqdm
import timeit
np.random.seed(1)

In [2]:
#
# LSH used to build LSH hash table
#

class LSH:
    def __init__(self, input_dim=3, hash_dim=1):
        self.planes = []
        for i in range(hash_dim):
            v = np.random.rand(input_dim)
            v_hat = v / np.linalg.norm(v)
            self.planes.append(v_hat)
    
        self.planes = np.matrix(self.planes)
        self.buckets = dict()
    
    # Returns LSH of a vector
    def hash(self, vector):
        hash_vector = np.where((self.planes @ vector) < 0, 1, 0)[0]
        
        hash_string = "".join([str(num) for num in hash_vector])
        return hash_string
    
    # Add vector to bucket
    def add(self, vector):
        hashed = self.hash(vector)
        
        if hashed in self.buckets:
            self.buckets[hashed].append(vector)
        else:
            self.buckets[hashed] = [vector]
    
    # Returns bucket vector is in
    def get(self, vector):
        hashed = self.hash(vector)
        
        if hashed in self.buckets:
            return self.buckets[hashed]
        
        return []
        

In [3]:
#
# Nearest Neighbour operations
#

class NN():
    
    # Returns Euclidean distance between vectors
    def _distance_(self, v1, v2):
        return np.linalg.norm(v1-v2)
    
    # Returns v1's Nearest Neighbour in vectors
    def get_nn(self, v1, vectors):
        min_dist = float("inf")
        min_vec = None
        
        for v2 in vectors:
            dist = self._distance_(v1, v2)
            if dist < min_dist:
                min_dist = dist
                min_vec = v2
        
        return min_vec    

In [4]:
input_dim = 726
lsh = LSH(input_dim=input_dim, hash_dim=6)

In [5]:
nn = NN()

In [6]:
vectors = []

for i in tqdm(range(1000000)):
    v = np.random.uniform(-1,1, [input_dim])
    lsh.add(v)
    vectors.append(v)

100%|██████████| 1000000/1000000 [01:08<00:00, 14683.22it/s]


In [7]:
v1  = np.random.uniform(-1,1, [input_dim])

bucket_vectors = lsh.get(v1)

print("Full vector list size: " + str(len(vectors)))
print("Bucket Size: " + str(len(bucket_vectors)) + " ("+ str(100 * len(bucket_vectors)/len(vectors)) +"%)")

# Find Nearest Neighbour in entire dataset w/ execution time
starttime = timeit.default_timer()
nn1 = nn.get_nn(v1, vectors)
print("Actual Nearest Neighbour: "  + ", time: " + str(timeit.default_timer() - starttime))

# Find Nearest Neighbour in LSH hash bucket w/ execution time
starttime = timeit.default_timer()
nn2 = nn.get_nn(v1, bucket_vectors)
print("Bucket Nearest Neighbour: " + ", time: " + str(timeit.default_timer() - starttime))

print(nn._distance_(nn1,nn2))

Full vector list size: 1000000
Bucket Size: 247421 (24.7421%)
Actual Nearest Neighbour: , time: 9.48607598200033
Bucket Nearest Neighbour: , time: 2.3659151309984736
0.0


In [8]:
print(nn._distance_(nn1,nn2))
print(nn._distance_(v1,nn2))
print(nn._distance_(nn1,v1))

21.302559324256993
20.594564353500193
19.93223626731763
